## Lane Finding Project

Guidelines on the code submission:
1. Code is developed using modular approach. Each stage used in lane detection algorithm is tagged with a function/method.
2. In the very end, a main() method is used to run the code.
3. To verify operation of code, test cases are added in the bottom part of this workbook. Each cell added against a test case simulates lane detection code for the corresponding test case.
4. For testing purpose, user is advised to run all cells by clicking on 'Cell' from menu bar and selecting 'Run All' from the drop down.
5. Uncomment cells given below for each test case to Run.
6. Reflections and analysis on the code is provided below test case section.

 ### Cell below contains all import code

In [2]:
import matplotlib.image as mpimg
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import Counter
from moviepy.editor import VideoFileClip
from IPython.display import HTML

### Function definitions go here

#### Reading an image

In [3]:
def readImg(path):
    image = mpimg.imread(path)
    #Returns the image
    return image
#plt.imshow(readImg(collectImages("test_images")[0]))
#plt.show()

#### Collecting image names from a folder

In [4]:
def collectImages(folderPath):
    # Returns array of strings with complete path of a file from current folder.
    return [folderPath + imageName for imageName in os.listdir(folderPath + "/")]
#print (collectImages("test_images"))

#### Convert RGB image to Gray Scale

In [99]:
def convertToGrayScale (rgbImage):
    grayImage = np.copy(rgbImage)
    mergedImage = cv2.cvtColor(grayImage, cv2.COLOR_RGB2GRAY)
    hsvImage = cv2.cvtColor(grayImage, cv2.COLOR_RGB2HSV)
    #Merges HSV image with RGB image
    mergedImage = cv2.addWeighted(grayImage, 0.8, hsvImage, 1, 0)
    mergedImage = cv2.cvtColor(mergedImage, cv2.COLOR_RGB2GRAY)
    
    # Returns 8 bit image array converted from input 32-bit (8+8+8) RGB image.
    return mergedImage
#plt.imshow(convertToGrayScale(readImg(collectImages("test_images/")[0])), cmap="gray")
#plt.show()

#### Apply Gaussian Blur

In [6]:
def applyGaussianBlur (image, kernelSize=3):
    filteredImage = np.copy(image)
    # Return image array after applying Gaussian blur. Image array size depends on input image.
    return cv2.GaussianBlur(filteredImage, (kernelSize, kernelSize), 0)
#plt.imshow(convertToGrayScale(readImg(collectImages("challenge-future-scope/")[0])), cmap="gray")
#plt.show()
#plt.imshow(applyGaussianBlur(convertToGrayScale(readImg(collectImages("challenge-future-scope/")[0])), 5), cmap="gray")
#plt.show()

#### Apply Canny Edge Detection

In [7]:
def cannyEdgeDetector (grayImage, lowThreshold=1, highThreshold=10):
    edges = cv2.Canny(grayImage, lowThreshold, highThreshold)
    # Returns 8-bit image array of size same as input gray scale image with boolean True for edges and False for non-edges
    return edges
#plt.imshow(convertToGrayScale(readImg(collectImages("challenge-future-scope/")[0])), cmap="gray")
#plt.show()
#plt.imshow(cannyEdgeDetector(applyGaussianBlur(convertToGrayScale(readImg(collectImages("challenge-future-scope/")[0])), 5), 50, 240), cmap="gray")
#plt.show()

#### Filter out region of Interest

In [8]:
def createPolyFilter (vertices):
    vertexArr = []
    for vertex in vertices:
        vertexArr.append((vertex[0], vertex[1]))
    vertexArr = np.array([vertexArr], dtype=np.int32)
    # Returns array of vertices as ordered pair tuples with data type as 32bit float
    return vertexArr
#print (createPolyFilter([(0, 0), (3,2), (400,322)]))

In [9]:
def applyRegionFilter (image, verticeArray):
    mask = np.zeros_like(image)
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
    #Create polygon and fill pixels inside polgon with ignore_mask_color
    cv2.fillPoly(mask, verticeArray, ignore_mask_color)
    maskedImage = cv2.bitwise_and(image, mask)
    # Returns image of same size as input image after applying region of interest filter
    return maskedImage
#plt.imshow(cannyEdgeDetector(applyGaussianBlur(convertToGrayScale(readImg(collectImages("challenge-future-scope/")[0])), 5), 175, 100), cmap="gray")
#plt.show()
#imagexsize = readImg(collectImages("challenge-future-scope/")[0]).shape[1]
#imageysize = readImg(collectImages("challenge-future-scope/")[0]).shape[0]
#plt.imshow(applyRegionFilter(cannyEdgeDetector(applyGaussianBlur(convertToGrayScale(readImg(collectImages("challenge-future-scope/")[0])), 5), 175, 100), \
#                           createPolyFilter([(int(imagexsize/20), imageysize),\
#                            (int(imagexsize/2), int(12*imageysize/20)),\
#                            (int(6*imagexsize/10), int(12*imageysize/20)),\
#                           (int(19*imagexsize/20), imageysize)])),\
#          cmap="gray")
#plt.show()

#### Draw Hough lines

In [10]:
def drawHoughLines (image, edgeImage, rho=1, theta=(np.pi/180), threshold=1, minLineLength=2, maxLineGap=5, extrapolateLines=False):
    houghLines = cv2.HoughLinesP(edgeImage, rho, theta, threshold, np.array([]), minLineLength, maxLineGap)
    
    if (extrapolateLines):
        houghLines = joinAndExtrapolateLineSegments(image, houghLines)
    
    return houghLines
#image = readImg(collectImages("challenge-future-scope/")[0])
#imagexsize = image.shape[1]
#imageysize = image.shape[0]
#edgeImage = applyRegionFilter(cannyEdgeDetector(applyGaussianBlur(convertToGrayScale(image), 5), 50, 240), \
#                           createPolyFilter([(int(imagexsize/20), imageysize),\
#                            (int(imagexsize/2), int(12*imageysize/20)),\
#                            (int(6*imagexsize/10), int(12*imageysize/20)),\
#                           (int(18*imagexsize/20), imageysize)]))
#plt.imshow(edgeImage, cmap="gray")
#plt.show()
#plt.imshow(drawLines(image, drawHoughLines(image, edgeImage, threshold=20, minLineLength=40, maxLineGap=80, extrapolateLines=True),\
#                     semiTransparent=True))
#plt.show()


#### Extrapolate lane lines

In [96]:
def joinAndExtrapolateLineSegments (image, lines):
    slopeRangeLeft = [0.3, 0.7]
    slopeRangeRight = [-0.5, -0.9]
    leftLines = []
    rightLines = []
    averageLines = []
    extrapolatedLine = []
    imageysize = image.shape[0]
    
    #Code for segregating into left and right lines
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope, intercept = np.polyfit([x1, x2], [y1, y2], deg=1)
            if (slope > slopeRangeLeft[0] and slope < slopeRangeLeft[1]):
                leftLines.append(line)
            elif (slope < slopeRangeRight[0] and slope > slopeRangeRight[1]):
                rightLines.append(line)

    #Calculate average of vectors for left line and add that line to averageLines. No action in case there was no line detected
    if (len(leftLines) > 0):
        averageLines.append(np.mean(leftLines, axis=0, dtype=np.int32))
    #Calculate average of vectors for Right line and add that line to averageLines. No action in case there was no line detected
    if (len(rightLines) > 0):
        averageLines.append(np.mean(rightLines, axis=0, dtype=np.int32))

    #Extrapolate the lines upto y value range of (imageysize/2, imageysize). Extrapolation is not done in X axis
    for line in averageLines:
        for x1, y1, x2, y2 in line:
            slope, intercept = np.polyfit([x1, x2], [y1, y2], deg=1)
            #line = np.array([int((25*imageysize/40 - intercept)/slope), 25*imageysize/40, int((imageysize - intercept)/slope), imageysize], dtype=np.int32)
            line = np.array([int((25*imageysize/40 - intercept)/slope), 25*imageysize/40, int((imageysize - intercept)/slope), imageysize], dtype=np.int32)
            extrapolatedLine.append([line])
            
    return extrapolatedLine
#image = readImg(collectImages("test_images/")[0])
#imagexsize = image.shape[1]
#imageysize = image.shape[0]
#edgeImage = applyRegionFilter(cannyEdgeDetector(applyGaussianBlur(convertToGrayScale(image), 5), 50, 200), \
#                           createPolyFilter([(int(imagexsize/20), 4*imageysize/5),\
#                            (int(imagexsize/2), int(12*imageysize/20)),\
#                            (int(6*imagexsize/10), int(12*imageysize/20)),\
#                           (int(18*imagexsize/20), imageysize)]))
#plt.imshow(edgeImage, cmap="gray")
#plt.show()
#plt.imshow(drawLines(image, drawHoughLines(image, edgeImage, threshold=20, minLineLength=20, maxLineGap=80, extrapolateLines=False),\
#                     semiTransparent=True))
#plt.show()

#### Draw lines on original image

In [12]:
def drawLines (image, lines, thickness=15, color=(255, 0, 0), semiTransparent=False):
    lineImage = np.copy(image)
    
    if (semiTransparent):
        blankImage = np.copy(image)*0
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(blankImage, (x1,y1), (x2,y2), color, thickness)
        lineImage = cv2.addWeighted(lineImage, 0.8, blankImage, 1, 0)
    else:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(lineImage, (x1,y1), (x2,y2), color, thickness)
    # Returns original image with lines detected by Hough and/or Extrapolated
    return lineImage

#### Apply Lane Finding algorithm on video clips

In [98]:
def laneDetectionInVideo (sourceFilePath, outputFilePath):
    originalVideoClip = VideoFileClip(sourceFilePath)
    laneDetectedClip = originalVideoClip.fl_image(processImage)
    %time laneDetectedClip.write_videofile(outputFilePath, audio=False)
#laneDetectionInVideo("challenge.mp4", "final-challenge-lane-detected.mp4")
#laneDetectionInVideo("solidYellowLeft.mp4", "final-yellow-lane-detected.mp4")
#laneDetectionInVideo("solidWhiteRight.mp4", "final-white-lane-detected-plain.mp4")


#### Process image method, combines all steps into one method

In [23]:
def processImage (image, laneLinesExtrapolated=True):
    imagexsize = image.shape[1]
    imageysize = image.shape[0]
    
    kernelSize = 5
    cannyLowThreshold = 50
    cannyHighThreshold = 200
    regionPolyVector = createPolyFilter([(int(imagexsize/20), imageysize),\
                            (int(imagexsize/2), int(12*imageysize/20)),\
                            (int(6*imagexsize/10), int(12*imageysize/20)),\
                           (int(19*imagexsize/20), imageysize)])

    houghThreshold = 20
    houghMinLineLenght = 40
    houghMaxLineGap = 80

    grayScaledImage = convertToGrayScale(image)
    gaussianBlurredImage = applyGaussianBlur(grayScaledImage, kernelSize)
    edgeImage = cannyEdgeDetector(gaussianBlurredImage, cannyLowThreshold, cannyHighThreshold)
    regionSelectedImage = applyRegionFilter(edgeImage, regionPolyVector)
    houghLines = drawHoughLines(image, regionSelectedImage, threshold=houghThreshold,\
                                         minLineLength=houghMinLineLenght, maxLineGap=houghMaxLineGap, \
                                extrapolateLines=laneLinesExtrapolated)
    imageWithLaneLines = drawLines(image, houghLines, thickness=10, semiTransparent=True)
    
    #Uncomment this line of code to plot image with lane detected markers
    #plt.imshow(imageWithLaneLines)
    #plt.show()

    return imageWithLaneLines
#image = readImg(collectImages("test_images/")[5])
#plt.imshow(process_image(image))
#plt.show()

### Main method to test code

In [15]:
def main (testFor="", laneLinesExtrapolated=True, optionalChallenge=False):
    if (testFor == "images"):
        pathToImageFolder = "test_images/"
        for imageName in collectImages(pathToImageFolder):
            if (imageName.find("output") == -1):
                image = readImg(imageName)
                laneDetectedImage = processImage(image, laneLinesExtrapolated)
                mpimg.imsave(imageName[0:imageName.rindex(".")] + "_output_lane_detected" + imageName[imageName.rindex("."):], laneDetectedImage)
    elif (testFor == "videos"):
        laneDetectionInVideo("solidWhiteRight.mp4", "solidWhiteRightLaneDetected.mp4")
        laneDetectionInVideo("solidYellowLeft.mp4", "solidYellowLeftLaneDetected.mp4")
        if (optionalChallenge):
            laneDetectionInVideo("challenge.mp4", "challengeLaneDetected.mp4")
    else:
        print ("Please specify the source ('images' or 'videos') to apply lane detection algorithm!!")
    
#main("images")

## Test Cases

To run Lane-Detection code for images and videos, follow steps given below:
    1. Click on 'Cell' from menu bar and select 'Run All' from the drop down.
    2. Uncomment cells given below for each test case to Run.

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

**Uncomment code in below cell to test on images**

In [101]:
#main("images", laneLinesExtrapolated=False)
#main("images")

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Uncomment code in below cell to test on videos**

In [17]:
#main("videos")

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

challenge.mp4

**Uncomment code in below cell to test on videos along with challenge video**

In [18]:
#main("videos", optionalChallenge=True)

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

**Lane finding algorithm fails in case following scenarios:**
1. Gray scale input to Canny edge detector fails where yellow colored lane lines are present and a glare of sunlight is seen on the road. This is because gray pixel mapping for version of yellow color is 226. Also, brighter portions of image map to gray scale value near 200. Hence, canny edge algorithm can't detect edges for yellow lanes.

2. When there is shadow casted onto the road. Shadow can be generated by tall buildings, trees, cars on other lanes, etc. Due to moving objects, shadow also moves and hence gets detected as foreground detail in an image.


**Algorithm can be made robust by:**
1. Merging Gray scale version and **HSV** version of original image and then supplying it to Canny Edge detector after smoothing. Yellow lanes will be more prominent in HSV image as yellow is warm color.

2. Removing shadow from images is the next big task. Shadows casting cannot be predicted and is seen a lot in city roads due to presence of high rise buildings. Background subtraction, shadow removal and pixel re-generation algorithms need to be used together to effectively remove shadows. Even in case of shadow, the illumination of region under shadow is similar to its surrounding area. This factor can also be taken into account.

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.